# 🚀 Google Colab Setup[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/06_reseaux_neurones_fondamentaux/06_exercices_solutions.ipynb)**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '06_exercices.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 06 - Solutions : Réseaux de Neurones (MLP)**Objectifs** :1. Implémenter des variantes de MLP2. Expérimenter avec hyperparamètres3. Régularisation et optimisation4. Comparaison NumPy vs PyTorch

In [ ]:
import numpy as npimport torchimport torch.nn as nnimport torch.optim as optimfrom torch.utils.data import DataLoader, TensorDatasetimport matplotlib.pyplot as pltfrom sklearn.datasets import make_classification, make_circles, fetch_openmlfrom sklearn.model_selection import train_test_splitfrom sklearn.preprocessing import StandardScalerfrom sklearn.metrics import accuracy_score, classification_reportimport seaborn as snsnp.random.seed(42)torch.manual_seed(42)

## Exercice 1 : Implémentation Perceptron simple (NumPy)**Objectif** : Implémenter un perceptron monocouche pour classification binaire.**TODO** :1. Implémenter la classe `Perceptron`2. Méthodes : `fit()`, `predict()`3. Tester sur dataset linéairement séparable4. Visualiser la frontière de décision

In [ ]:
class Perceptron:    """Perceptron simple pour classification binaire."""        def __init__(self, learning_rate=0.01, n_epochs=100):        self.lr = learning_rate        self.n_epochs = n_epochs        self.weights = None        self.bias = None        def fit(self, X, y):        """Entraîne le perceptron."""        # SOLUTION: Implémenter l'algorithme du perceptron        # - Initialiser weights et bias        # - Pour chaque epoch:        #     - Pour chaque échantillon:        #         - Calculer prédiction (z = X @ w + b, pred = 1 if z >= 0 else 0)        #         - Mettre à jour poids si erreur: w += lr * (y - pred) * X        pass        def predict(self, X):        """Prédiction (0 ou 1)."""        # SOLUTION: Implémenter la prédiction        pass# Test sur dataset linéairement séparableX, y = make_classification(n_samples=200, n_features=2, n_redundant=0,                            n_informative=2, n_clusters_per_class=1, random_state=42)X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# SOLUTION: Entraîner le perceptron et évaluer# perceptron = Perceptron(learning_rate=0.01, n_epochs=100)# perceptron.fit(X_train, y_train)# y_pred = perceptron.predict(X_test)# print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")# SOLUTION: Visualiser la frontière de décision

## Exercice 2 : XOR Problem avec MLP**Contexte** : Le perceptron simple ne peut pas résoudre XOR (non linéairement séparable).**TODO** :1. Créer dataset XOR2. Entraîner MLP PyTorch avec une couche cachée3. Comparer avec perceptron simple4. Visualiser la frontière de décision

In [ ]:
# Dataset XORX_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)y_xor = np.array([0, 1, 1, 0], dtype=np.int64)print("Dataset XOR:")for x, y in zip(X_xor, y_xor):    print(f"Input: {x} -> Output: {y}")# SOLUTION: Créer MLP PyTorch pour résoudre XOR# Architecture suggérée: 2 -> 4 -> 1 (ou 2 -> 4 -> 2 avec Softmax)# Indices:# - Utiliser nn.Sequential# - Activation: ReLU ou Sigmoid# - Optimizer: Adam ou SGD# - Loss: BCEWithLogitsLoss ou CrossEntropyLoss# - Entraîner sur ~1000 epochs# SOLUTION: Tester si le MLP prédit correctement les 4 cas

## Exercice 3 : Impact du Dropout**Objectif** : Comparer MLP avec et sans Dropout pour comprendre l'effet de régularisation.**TODO** :1. Entraîner 2 modèles sur MNIST (avec/sans Dropout)2. Comparer overfitting (train vs val accuracy)3. Visualiser les courbes d'apprentissage4. Conclure sur l'efficacité du Dropout

In [ ]:
# Chargement MNIST (petit subset)mnist = fetch_openml('mnist_784', version=1, parser='auto')X = mnist.data.astype('float32').values[:5000] / 255.0  # type: ignorey = mnist.target.astype('int').values[:5000]  # type: ignoreX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# SOLUTION: Créer 2 modèles identiques sauf Dropout# Modèle 1: 784 -> 256 -> 128 -> 10 (sans Dropout)# Modèle 2: 784 -> 256 -> Dropout(0.3) -> 128 -> Dropout(0.3) -> 10# SOLUTION: Entraîner les 2 modèles sur 50 epochs# SOLUTION: Comparer:# - Train accuracy finale# - Val accuracy finale# - Gap train-val (indicateur d'overfitting)# SOLUTION: Visualiser les courbes d'apprentissage côte à côte

## Exercice 4 : Comparaison d'Optimizers**Objectif** : Comparer SGD, SGD+Momentum, Adam sur même architecture.**TODO** :1. Entraîner 3 modèles identiques avec optimizers différents2. Comparer vitesse de convergence3. Comparer accuracy finale4. Visualiser les courbes de loss

In [ ]:
# Dataset: Make Circles (non linéaire)X, y = make_circles(n_samples=1000, noise=0.1, factor=0.5, random_state=42)X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# Conversion PyTorchX_train_t = torch.FloatTensor(X_train)y_train_t = torch.LongTensor(y_train)X_test_t = torch.FloatTensor(X_test)y_test_t = torch.LongTensor(y_test)train_dataset = TensorDataset(X_train_t, y_train_t)train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)# SOLUTION: Créer 3 modèles identiques (2 -> 32 -> 16 -> 2)# SOLUTION: Définir 3 optimizers:# - SGD (lr=0.1)# - SGD + Momentum (lr=0.1, momentum=0.9)# - Adam (lr=0.01)# SOLUTION: Entraîner les 3 modèles sur 100 epochs# Stocker l'historique de loss pour chaque modèle# SOLUTION: Comparer les 3 optimizers:# - Courbes de loss (même graphique)# - Accuracy finale sur test set# - Nombre d'epochs pour atteindre 95% accuracy (si atteint)

## Exercice 5 : Batch Normalization vs Layer Normalization**Objectif** : Comprendre l'effet de BatchNorm sur la stabilité de l'entraînement.**TODO** :1. Entraîner 3 modèles:   - Sans normalisation   - Avec Batch Normalization   - Avec Layer Normalization2. Comparer stabilité et vitesse de convergence3. Tester sur architecture profonde (5 couches cachées)

In [ ]:
# SOLUTION: Créer 3 architectures profondes:# Architecture: 784 -> 512 -> 256 -> 128 -> 64 -> 32 -> 10# Modèle 1: Sans normalisation# model1 = nn.Sequential(#     nn.Linear(784, 512), nn.ReLU(),#     nn.Linear(512, 256), nn.ReLU(),#     ...# )# Modèle 2: Avec BatchNorm après chaque Linear# model2 = nn.Sequential(#     nn.Linear(784, 512), nn.BatchNorm1d(512), nn.ReLU(),#     ...# )# Modèle 3: Avec LayerNorm# model3 = nn.Sequential(#     nn.Linear(784, 512), nn.LayerNorm(512), nn.ReLU(),#     ...# )# SOLUTION: Entraîner les 3 modèles sur MNIST (50 epochs)# SOLUTION: Comparer:# - Convergence (loss après 10 epochs)# - Stabilité (variance de la loss)# - Accuracy finale# SOLUTION: Visualiser l'évolution de la loss pour les 3 modèles

## Exercice 6 : Learning Rate Scheduling**Objectif** : Expérimenter avec différents schedulers pour optimiser la convergence.**TODO** :1. Tester 4 strategies:   - Learning rate constant   - StepLR (réduction par paliers)   - ExponentialLR (réduction exponentielle)   - ReduceLROnPlateau (réduction adaptative)2. Comparer vitesse de convergence3. Visualiser l'évolution du learning rate

In [ ]:
# SOLUTION: Créer 4 modèles identiques# SOLUTION: Définir 4 schedulers:# 1. Pas de scheduler (lr constant = 0.1)# 2. StepLR: lr *= 0.5 tous les 10 epochs# 3. ExponentialLR: lr *= 0.95 chaque epoch# 4. ReduceLROnPlateau: lr *= 0.5 si val_loss stagne 5 epochs# SOLUTION: Entraîner les 4 modèles sur 50 epochs# Stocker: loss, accuracy, learning_rate# SOLUTION: Visualiser:# - Graphique 1: Loss vs Epoch (4 courbes)# - Graphique 2: Learning Rate vs Epoch (4 courbes)# - Graphique 3: Test Accuracy finale (bar plot)# SOLUTION: Conclure sur la meilleure stratégie

## Exercice 7 : Weight Initialization**Objectif** : Comprendre l'impact de l'initialisation des poids.**TODO** :1. Tester 4 méthodes d'initialisation:   - Zeros (tous les poids à 0)   - Random uniforme [-1, 1]   - Xavier/Glorot   - He/Kaiming2. Comparer vitesse de convergence3. Analyser la distribution des gradients

In [ ]:
def init_weights_zeros(m):    if isinstance(m, nn.Linear):        m.weight.data.fill_(0.0)        m.bias.data.fill_(0.0)def init_weights_uniform(m):    if isinstance(m, nn.Linear):        nn.init.uniform_(m.weight, -1, 1)        nn.init.uniform_(m.bias, -1, 1)def init_weights_xavier(m):    if isinstance(m, nn.Linear):        nn.init.xavier_uniform_(m.weight)        nn.init.zeros_(m.bias)def init_weights_he(m):    if isinstance(m, nn.Linear):        nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')        nn.init.zeros_(m.bias)# SOLUTION: Créer 4 modèles identiques et appliquer les initialisations# model1.apply(init_weights_zeros)# model2.apply(init_weights_uniform)# model3.apply(init_weights_xavier)# model4.apply(init_weights_he)# SOLUTION: Entraîner les 4 modèles et comparer:# - Loss après 5 epochs (indicateur de vitesse initiale)# - Accuracy finale après 50 epochs# - Distribution des poids après initialisation (histogramme)# SOLUTION: Visualiser l'évolution de la loss pour chaque initialisation

## Exercice 8 : Mini-Projet - Fashion MNIST**Objectif** : Appliquer toutes les techniques apprises sur Fashion MNIST.**TODO** :1. Charger Fashion MNIST (10 classes de vêtements)2. Construire architecture optimale (expérimenter)3. Appliquer les meilleures pratiques:   - BatchNorm   - Dropout   - Adam optimizer   - Learning rate scheduler   - Early stopping4. Atteindre >88% accuracy sur test set5. Analyser les erreurs de classification

In [ ]:
# Chargement Fashion MNISTfashion_mnist = fetch_openml('Fashion-MNIST', version=1, parser='auto')X_fashion = fashion_mnist.data.astype('float32').values / 255.0  # type: ignorey_fashion = fashion_mnist.target.astype('int').values  # type: ignore# Labels Fashion MNISTclass_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']# SOLUTION: Split train/val/test# SOLUTION: Construire votre meilleure architecture MLP# Suggestions:# - 3-4 couches cachées# - BatchNorm après chaque couche# - Dropout 0.3-0.5# - ReLU activation# - Adam optimizer (lr=0.001)# - ReduceLROnPlateau scheduler# SOLUTION: Entraîner avec early stopping# SOLUTION: Évaluation complète:# - Accuracy par classe# - Matrice de confusion# - Classification report# - Visualisation des erreurs# SOLUTION: Analyser:# - Quelles classes sont les plus difficiles?# - Quelles confusions sont fréquentes? (ex: Shirt vs T-shirt)# - Visualiser 10 prédictions incorrectes

## Solutions et Indices### Exercice 1 - Perceptron```python# Initialisationself.weights = np.random.randn(X.shape[1]) * 0.01self.bias = 0.0# Update rulefor epoch in range(self.n_epochs):    for xi, yi in zip(X, y):        z = np.dot(xi, self.weights) + self.bias        pred = 1 if z >= 0 else 0        error = yi - pred        self.weights += self.lr * error * xi        self.bias += self.lr * error```### Exercice 2 - XOR avec MLP```pythonmodel = nn.Sequential(    nn.Linear(2, 4),    nn.ReLU(),    nn.Linear(4, 2))optimizer = optim.Adam(model.parameters(), lr=0.01)criterion = nn.CrossEntropyLoss()```### Exercice 3 - Dropout- Avec Dropout: moins d'overfitting, meilleure généralisation- Sans Dropout: train accuracy > val accuracy (signe d'overfitting)### Exercice 4 - Optimizers- **SGD** : Convergence lente, peut osciller- **SGD + Momentum** : Convergence plus rapide, moins d'oscillations- **Adam** : Convergence la plus rapide, adapte le learning rate### Exercice 8 - Fashion MNISTArchitecture suggérée:```pythonmodel = nn.Sequential(    nn.Linear(784, 512),    nn.BatchNorm1d(512),    nn.ReLU(),    nn.Dropout(0.4),        nn.Linear(512, 256),    nn.BatchNorm1d(256),    nn.ReLU(),    nn.Dropout(0.3),        nn.Linear(256, 128),    nn.BatchNorm1d(128),    nn.ReLU(),    nn.Dropout(0.2),        nn.Linear(128, 10))```